In [ ]:
!pip install scikit-learn
!pip install transformers
!pip install datasets
!pip install torch
!pip install rouge_score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=58d5dd948c9cc4551c51b3c6

In [ ]:
import pandas as pd
df = pd.read_csv('news.csv')
print(df.head())

                                               title  \
0   Садыр Жапаров Өзгөн районунда айланма жолду ачты   
1  Тай-Мурас Ташиев «Өзгөн айланма жолу» долбоору...   
2        7-декабрга карата өлкөдө күтүлгөн аба ырайы   
3  Салык кызматынын төрагасы жана анын орун басар...   
4  Окуучулардын кышкы каникулу 30-декабрдан башталат   

                                                 url             date  \
0  https://serep.kg/https://serep.kg/kabarlar/say...  10:2707.12.2024   
1  https://serep.kg/https://serep.kg/kabarlar/koo...  10:0307.12.2024   
2  https://serep.kg/https://serep.kg/kabarlar/koo...  18:0006.12.2024   
3  https://serep.kg/https://serep.kg/kabarlar/kyl...  17:4406.12.2024   
4  https://serep.kg/https://serep.kg/kabarlar/koo...  17:4006.12.2024   

                                             content  \
0  “Өзүңүздөргө белгилүү болгондой, ушул жылдын я...   
1  «Мен «Өзгөн айланма жолу» долбоорунан чыктым. ...   
2  Эртең, 7-декабрда Чүй, Ысык-Көл облустарынын а...   


In [ ]:
df.dropna(subset=['content', 'summary'], inplace=True)
df.drop_duplicates(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(
    df,
    test_size=0.1,  # 10% for validation
    random_state=42
)

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

Train size: 2356, Validation size: 262


In [ ]:
model_checkpoint = "facebook/mbart-large-50-many-to-many-mmt"

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
from transformers import MBart50TokenizerFast

tokenizer = MBart50TokenizerFast.from_pretrained(model_checkpoint)
tokenizer.src_lang = "ky_KG"
tokenizer.tgt_lang = "ky_KG"

max_input_length = 512
max_target_length = 80

def preprocess_function(examples):
    inputs = examples['content']
    targets = examples['summary']

    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset_tokenized = train_dataset.map(preprocess_function, batched=True)
val_dataset_tokenized   = val_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Map:   0%|          | 0/2356 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/262 [00:00<?, ? examples/s]

In [ ]:
from transformers import MBartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

model = MBartForConditionalGeneration.from_pretrained(model_checkpoint)

training_args = Seq2SeqTrainingArguments(
    output_dir="./mbart-ky",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=500,
    num_train_epochs=6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    fp16=True,
)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=val_dataset_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)
trainer.train()

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-c2617c005221>:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,3.977700,3.100986
2,2.815600,2.690962
3,2.252400,2.488552
4,1.896800,2.422751
5,1.584800,2.373834
6,1.228400,2.379286


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3534, training_loss=2.1508083656539876, metrics={'train_runtime': 2788.5811, 'train_samples_per_second': 5.069, 'train_steps_per_second': 1.267, 'total_flos': 9232252167979008.0, 'train_loss': 2.1508083656539876, 'epoch': 6.0})

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

def compute_rouge(decoded_preds, decoded_labels):
    rouge1, rouge2, rougeL = 0.0, 0.0, 0.0

    for pred, ref in zip(decoded_preds, decoded_labels):
        scores = scorer.score(pred, ref)
        rouge1 += scores['rouge1'].fmeasure
        rouge2 += scores['rouge2'].fmeasure
        rougeL += scores['rougeL'].fmeasure

    n = len(decoded_preds)
    return {
        'rouge1': (rouge1 / n) * 100,
        'rouge2': (rouge2 / n) * 100,
        'rougeL': (rougeL / n) * 100
    }

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [
        [(l if l != -100 else tokenizer.pad_token_id) for l in label]
        for label in labels
    ]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = compute_rouge(decoded_preds, decoded_labels)
    return result

trainer.compute_metrics = compute_metrics
eval_result = trainer.evaluate()
print(eval_result)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 2.379286289215088, 'eval_rouge1': 30.309566387810666, 'eval_rouge2': 8.09231904651752, 'eval_rougeL': 29.48283904390775, 'eval_runtime': 169.0829, 'eval_samples_per_second': 1.55, 'eval_steps_per_second': 0.39, 'epoch': 6.0}


**{'eval_loss': 2.379286289215088, 'eval_rouge1': 30.309566387810666, 'eval_rouge2': 8.09231904651752, 'eval_rougeL': 29.48283904390775, 'eval_runtime': 169.0829, 'eval_samples_per_second': 1.55, 'eval_steps_per_second': 0.39, 'epoch': 6.0}**

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
def generate_summary_kyrgyz(article_text, max_length=80):
    inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    summary_ids = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=4,
        early_stopping=True,
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:

text1 = """Саламаттык сактоо министри Алымкадыр Бейшеналиев Казакстандагы авиакырсыктан жабыркаган кыргызстандыктардан кабар алып, ден соолугунун абалы менен таанышты.

Ведомстводон билдиришкендей,  жабырлануучулардын экөөнүн абалы канааттандырарлык деп бааланып, Улуттук госпиталдын кыска мөөнөттүү стационардык бөлүмүндө жатат.

Үчүнчү бейтаптын абалы туруктуу жана оор. Саламаттыкты сактоо министрлигинин башчысы Алымкадыр Бейшеналиевдин катышуусунда дарыгерлердин кеңешмеси өттү. Бейтап Улуттук госпиталдын реанимация бөлүмүндө адистердин такай көзөмөлүндө болуп, бардык зарыл болгон дарыланууда.

Эскерте кетсек, 25-декабрда Баку-Грозный багыты боюнча учкан Азербайжан аба жолдорунун учагы Актау аэропортуна жакын жерде кулап түшкөн. Учак Орусияга конуу аракети ийгиликсиз болгондон кийин шашылыш түрдө ал жакка багыт алган. Кырсыктын кесепетинен 38 адам каза болуп, 29 аман калганы ооруканага жаткырылган. Жабыркагандардын арасында Кыргызстандын үч жараны бар болгон."""
print(generate_summary_kyrgyz(text1))


Алымкадыр Бейшеналиев Казакстандагы жабыркаган жарандардын абалы менен таанышты. Жабырлануучулардын абалы канааттандырарлык деп бааланууда. Алымкадыр бейшеналиевден дарыгерлер жана адистер катышты.


In [ ]:
text2 = """Бүгүн, 30-декабрда министрлер кабинетинин быйылкы акыркы жыйынында өлкөнүн ар кайсы аймактарында жайгашкан 17 жаңы өнөр жай жана инфраструктуралык объектилердин салтанаттуу ачылышы болду. Салтанат онлайн режиминде өттү.

Бул объекттер 2024-жылга карата жаңы өндүрүштүк кубаттуулуктарды жана инфраструктуралык долбоорлорду ишке киргизүү программасынын акыркы объекттери болгон. Жалпысынан быйылкы жылдын программасында 102 объектти куруу пландалууда.

Кабинеттин башчысы Адылбек Касымалиев келерки жылы алардын саны эки эсеге көбөйүшү керектигин белгиледи."""

print(generate_summary_kyrgyz(text2))

Министрлер кабинетинин 17 жаңы көч жай объекти боюнча салтанаттуу ачылышы өттү. Бул объекттер 2024-жылга карата жаңы көчтүк кубаттуулуктар болгон. Жалпысынан 102 объект 2024-жылы пландалууда.


In [ ]:
text3 = """
Бириккен Араб Эмиратында Кыргызстандын жараны дайынсыз жоголду. Бул тууралуу Тышкы иштер министрлигинин басма сөз кызматы билдирди.

Маалым болгондой, Кыргызстандын Дубай шаарындагы Башкы консулдугу дайынсыз жоголгон КР жараны А.Б.ны издөөгө көмөк көрсөтүү өтүнүчү менен жергиликтүү ыйгарым укуктуу органдарга кайрылган.

Дайынсыз болуп жаткан мекендештин жакындарынан алынган маалыматка караганда, ал Дубайдын аэропортунда болгон жана учуп кетүүнү пландаган", - деп айтылат маалыматта.

Башкы консулдуктун кызматкерлери дароо аэропортко барышкан, тилекке каршы, андан майнап чыккан эмес.

Ошондой эле Башкы консулдуктун кызматкерлери Дубайдын иммиграция департаментинен кыргызстандыктын Эмираттын аймагынан чыкпаганын аныкташкан.

"Кыргыз Республикасынын Башкы консулдугу А.Б.ны издөө боюнча зарыл иш-чараларды улантууда жана Дубайдын ыйгарым укуктуу органдары, ошондой эле дайынсыз жоголгон жарандын жакындары менен байланышта", - деп айтылат маалыматта.
"""

print(generate_summary_kyrgyz(text3))

Бириккен Араб Эмиратында Кыргызстандын жараны дайынсыз жоголду. Жаран дайынсыз жоголуп, Дубайдын аэропортунда болгон жана учуп кетүүгө пландалууда. Дубайдын иммиграция департаменти менен Эмираттын аймагынан чыкпаганын аныкташкан.


In [ ]:
text4 = """Бүгүн, 7-октябрда өлкө аймагы боюнча 2025-жылдагы ажылыкка баруучуларды каттоо башталган. Муфтияттан кабарлашкандай, саат 13:00гө карата квота толуп, ажылыкка төлөм кабыл алуу жабылды.

Белгилесек, 2025-жылы ажылыкка барууну каалаган талапкерлерди өлкө боюнча бир тизмеге (жандуу тизме) каттоо электрондук (Eid) ID-карт паспортунун негизинде жүргүзүлөт. Талап боюнча, каттоолор баштапкы жана кайтарымсыз акчаны РСК банкына төлөп, Муфтияттын түзүмүндөгү облустук, шаардык Казыяттарга жана райондук Хатибияттарга дүмүрчөктү жана электрондук каттоого тиешелүү иш кагаздарды тапшыруусу зарыл. Талапкерлер баштапкы төлөмдөрдү аткарбай туруп электрондук базага каттала алышпайт.

Быйыл Сауд Арабиясы 2025-жылдагы ажылык сапарга Кыргызстандан баруучу зыяратчыларга 6 миң квота берген.

"""

print(generate_summary_kyrgyz(text4))

Бишкекте 2025-жылы ажылыкка баруучуларды каттоо жабылды. Быр тизмеге ID-карта үчүн төлөмдөр жабылды. Талапкерлер 2025-жылы ажылык сапарга 6 миң квота берген.


In [ ]:
text5 = """Түштүк Кореяда кызматтан четтетилген президент Юн Сок Ёлду камакка алууга барган тергөөчүлөр менен полиция кызматкерлеринин аракети ишке ашпады. Соттун ордери менен келген тергөө тобу менен президенттин тарапташтары 3-январь күнү алты саат бою тирешип, акыры аны камакка алуу операциясын токтотууга мажбур болушту.

Президенттин коопсуздук кызматы менен аны колдоого келген тарапташтары бул аракетти мыйзамдан аша чабуу катары сыпатташкан. Юн Сок Ёл резиденциясынын алдына чогулган тарапташтарына карата жаңы жылдык кайрылуусунда “антимамлекеттик күчтөргө” каршы аягына чейин күрөшөрүн билдирген. Президенттик резиденциянын алдында бир нече күндөн бери чогулуп турган анын тарапташтары бул күндү жеңиш катары баалашкан.

Оппозициялык күчтөр болсо парламент президент Юн Сок Ёлго импичмент жарыялагандан кийин өлкө жетекчилигинин милдетин аткарып жаткан премьер-министрдин орун басары Чхве Сан Мокту президенттин коопсуздук кызматынын иш-аракеттерин токтотууга аракеттенүүгө чакырып, учурдагы абалды жөнгө салуу жоопкерчилиги анын мойнунда экенин эскертишкен.

Тергөө тобу президентти камакка алуу мүмкүн эмес экенин, кырдаалды талдап чыккандан кийин кандайдыр бир кадамдары анык болорун билдирди.

31-декабрь күнү Сеулдун соту импичмент менен кызматтан четтетилген президент Юн Сок Ёлду камакка алууга ордер берген. Буга анын суракка келүүдөн баш тартканы себеп болгон.

Юн Сок Ёл өткөн жылдын 3-декабрында Түштүк Кореяда аскердик абал жарыялаган. Ал өз чечимин "оппозициянын мамлекетке каршы ишмердүүлүгү өкмөткө жолтоо болуп жатканы" менен түшүндүргөн.

Курч тирешүүлөрдөн кийин парламенттеги оппозициялык күчтөр аскердик абалды жокко чыгарууга жетишкен. Арадан бир аз убакыт өткөн соң, 14-декабрда президентке импичмент жарыяланып, анын ыйгарым укуктары убактылуу токтотулган.

Конституциялык сот 180 күндүн ичинде аны кызматынан биротоло ажыратууну же ыйгарым укуктарын калыбына келтирүүнү чечүүгө тийиш. Учурда Юн Сок Ёлго өлкөдөн чыгууга тыюу салынган."""

print(generate_summary_kyrgyz(text5))

Түштүк Кореяда президент Юн Сок Ёлду камакка алуу операциясын токтотууга мажбур болушту. Ал бул чечимти мыйзамдан аша чабуу катары сыпатташкан. Президенттик резиденциянын алдында бир нече кызматты жеңиш катары баалашкан.
